# Llama 3 Cookbook with LlamaIndex and Groq

<a href="https://colab.research.google.com/github/meta-llama/llama-recipes/blob/main/recipes/llama_api_providers/llama3_cookbook_groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Meta developed and released the Meta [Llama 3](https://ai.meta.com/blog/meta-llama-3/) family of large language models (LLMs), a collection of pretrained and instruction tuned generative text models in 8 and 70B sizes. The Llama 3 instruction tuned models are optimized for dialogue use cases and outperform many of the available open source chat models on common industry benchmarks.

In this notebook, we demonstrate how to use Llama 3 with LlamaIndex for a comprehensive set of use cases. 
1. Basic completion / chat 
2. Basic RAG (Vector Search, Summarization)
3. Advanced RAG (Routing)
4. Text-to-SQL 
5. Structured Data Extraction
6. Chat Engine + Memory
7. Agents


We use Llama3-8B and Llama3-70B through [Groq](https://groq.com) - you can sign up there to get a free trial API key.

## Installation and Setup

In [1]:
!pip install llama-index
!pip install llama-index-llms-groq
!pip install llama-index-embeddings-huggingface
!pip install llama-parse

You should consider upgrading via the '/Users/daniel/.pyenv/versions/3.10.3/bin/python3.10 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/daniel/.pyenv/versions/3.10.3/bin/python3.10 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/daniel/.pyenv/versions/3.10.3/bin/python3.10 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/daniel/.pyenv/versions/3.10.3/bin/python3.10 -m pip install --upgrade pip' command.


In [2]:
import nest_asyncio

nest_asyncio.apply()

### Setup LLM using Groq

To use [Groq](https://groq.com), you need to make sure that `GROQ_API_KEY` is specified as an environment variable.

In [3]:
import os

os.environ["GROQ_API_KEY"] = "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"

In [4]:
from llama_index.llms.groq import Groq

llm = Groq(model="llama3-8b-8192")
llm_70b = Groq(model="llama3-70b-8192")

### Setup Embedding Model

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

### Define Global Settings Configuration

In LlamaIndex, you can define global settings so you don't have to pass the LLM / embedding model objects everywhere.

In [6]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

### Download Data

Here you'll download data that's used in section 2 and onwards.

We'll download some articles on Kendrick, Drake, and their beef (as of May 2024).

In [7]:
!mkdir data
!wget "https://www.dropbox.com/scl/fi/t1soxfjdp0v44an6sdymd/drake_kendrick_beef.pdf?rlkey=u9546ymb7fj8lk2v64r6p5r5k&st=wjzzrgil&dl=1" -O data/drake_kendrick_beef.pdf
!wget "https://www.dropbox.com/scl/fi/nts3n64s6kymner2jppd6/drake.pdf?rlkey=hksirpqwzlzqoejn55zemk6ld&st=mohyfyh4&dl=1" -O data/drake.pdf
!wget "https://www.dropbox.com/scl/fi/8ax2vnoebhmy44bes2n1d/kendrick.pdf?rlkey=fhxvn94t5amdqcv9vshifd3hj&st=dxdtytn6&dl=1" -O data/kendrick.pdf

mkdir: data: File exists
--2024-05-20 09:27:56--  https://www.dropbox.com/scl/fi/t1soxfjdp0v44an6sdymd/drake_kendrick_beef.pdf?rlkey=u9546ymb7fj8lk2v64r6p5r5k&st=wjzzrgil&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6019:18::a27d:412, 162.125.4.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6019:18::a27d:412|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4425830a1d2d4c42bbf6c89b7f.dl.dropboxusercontent.com/cd/0/inline/CTQhAFm1iI5gNTeE_NytPzfcLl6Ilp9PSwNsVHJg7h_C2mUfnd6DL__txef3V5PoEV68APiuzt1UaHr4GVFHs-iYtSYqNJ9YT-chZyGn5GTRT837J92mPPDHpPnxibg3FCE/file?dl=1# [following]
--2024-05-20 09:27:57--  https://uc4425830a1d2d4c42bbf6c89b7f.dl.dropboxusercontent.com/cd/0/inline/CTQhAFm1iI5gNTeE_NytPzfcLl6Ilp9PSwNsVHJg7h_C2mUfnd6DL__txef3V5PoEV68APiuzt1UaHr4GVFHs-iYtSYqNJ9YT-chZyGn5GTRT837J92mPPDHpPnxibg3FCE/file?dl=1
Resolving uc4425830a1d2d4c42bbf6c89b7f.dl.dropboxusercontent.com (uc4425830a1d2d4c42bbf6c89b7f.dl.dropboxuserc

### Load Data

We load data using LlamaParse by default, but you can also choose to opt for our free pypdf reader (in SimpleDirectoryReader by default) if you don't have an account! 

1. LlamaParse: Signup for an account here: cloud.llamaindex.ai. You get 1k free pages a day, and paid plan is 7k free pages + 0.3c per additional page. LlamaParse is a good option if you want to parse complex documents, like PDFs with charts, tables, and more. 

2. Default PDF Parser (In `SimpleDirectoryReader`). If you don't want to signup for an account / use a PDF service, just use the default PyPDF reader bundled in our file loader. It's a good choice for getting started!

In [8]:
# Uncomment this code if you want to use LlamaParse
# from llama_parse import LlamaParse

# docs_kendrick = LlamaParse(result_type="text").load_data("./data/kendrick.pdf")
# docs_drake = LlamaParse(result_type="text").load_data("./data/drake.pdf")
# docs_both = LlamaParse(result_type="text").load_data(
#     "./data/drake_kendrick_beef.pdf"
# )

# Uncomment this code if you want to use SimpleDirectoryReader / default PDF Parser
from llama_index.core import SimpleDirectoryReader

docs_kendrick = SimpleDirectoryReader(input_files=["data/kendrick.pdf"]).load_data()
docs_drake = SimpleDirectoryReader(input_files=["data/drake.pdf"]).load_data()
docs_both = SimpleDirectoryReader(input_files=["data/drake_kendrick_beef.pdf"]).load_data()

## 1. Basic Completion and Chat

### Call complete with a prompt

In [9]:
response = llm.complete("do you like drake or kendrick better?")

print(response)

I'm just an AI, I don't have personal preferences or opinions, nor do I have the capacity to enjoy or dislike music. I can provide information and insights about different artists and their work, but I don't have personal feelings or emotions.

However, I can tell you that both Drake and Kendrick Lamar are highly acclaimed and influential artists in the music industry. They have both received widespread critical acclaim and have won numerous awards for their work.

Drake is known for his introspective and emotive lyrics, as well as his ability to blend different genres such as hip-hop, R&B, and pop. He has released several successful albums, including "Take Care" and "Views".

Kendrick Lamar is known for his socially conscious and thought-provoking lyrics, as well as his unique blend of jazz, funk, and hip-hop. He has released several critically acclaimed albums, including "Good Kid, M.A.A.D City" and "To Pimp a Butterfly".

Ultimately, whether you prefer Drake or Kendrick Lamar depend

In [10]:
stream_response = llm.stream_complete(
    "you're a drake fan. tell me why you like drake more than kendrick"
)

for t in stream_response:
    print(t.delta, end="")

Man, I'm a die-hard Drake fan, and I gotta say, I love the 6 God for a lot of reasons. Now, I know some people might say Kendrick is the king of hip-hop, and I respect that, but for me, Drake brings something unique to the table that sets him apart.

First of all, Drake's lyrics are so relatable. He's not just rapping about gangsta life or street cred; he's talking about real-life struggles, relationships, and emotions. His songs are like a diary entry, you know? He's sharing his thoughts, feelings, and experiences in a way that resonates with people from all walks of life. I mean, who hasn't been through a breakup or felt like they're stuck in a rut? Drake's music speaks to that.

And let's not forget his storytelling ability. The man can paint a picture with his words. He's got this effortless flow, and his rhymes are like a puzzle – intricate, clever, and always surprising. He's got this ability to weave together complex narratives that keep you engaged from start to finish.

Now, I

### Call chat with a list of messages

In [11]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are Kendrick."),
    ChatMessage(role="user", content="Write a verse."),
]
response = llm.chat(messages)

In [12]:
print(response)

assistant: "I'm the king of the game, no debate
My rhymes are fire, can't nobody relate
I'm on a mission, to spread the message wide
My flow's on a hundred, ain't nobody gonna divide"


## 2. Basic RAG (Vector Search, Summarization)

### Basic RAG (Vector Search)

In [13]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(docs_both)
query_engine = index.as_query_engine(similarity_top_k=3)

In [14]:
response = query_engine.query("Tell me about family matters")

In [15]:
print(str(response))

Drake's diss track "Family Matters" is essentially three songs in one, on three different beats. The track is a seven-and-a-half-minute diss track with an accompanying video.


### Basic RAG (Summarization)

In [16]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(docs_both)
summary_engine = summary_index.as_query_engine()

In [17]:
response = summary_engine.query(
    "Given your assessment of this article, who won the beef?"
)

In [18]:
print(str(response))

It's difficult to declare a clear winner in this beef, as both parties have delivered strong diss tracks and have been engaging in a back-and-forth exchange.


## 3. Advanced RAG (Routing)

### Build a Router that can choose whether to do vector search or summarization

In [19]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts.",
    ),
)

summary_tool = QueryEngineTool(
    index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document.",
    ),
)

In [20]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool], select_multi=False, llm=llm_70b
)

response = query_engine.query(
    "Tell me about the song meet the grahams - why is it significant"
)

In [21]:
print(response)

"Meet the Grahams" is significant because it marks a turning point in the beef between Kendrick Lamar and Drake. The song is notable for its lighthearted and humorous tone, with Kendrick cracking jokes and making playful jabs at Drake. The track also showcases Kendrick's ability to poke fun at himself and not take himself too seriously.


## 4. Text-to-SQL 

Here, we download and use a sample SQLite database with 11 tables, with various info about music, playlists, and customers. We will limit to a select few tables for this test.

In [22]:
!wget "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip" -O "./data/chinook.zip"
!unzip "./data/chinook.zip"

--2024-05-20 09:31:46--  https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Resolving www.sqlitetutorial.net (www.sqlitetutorial.net)... 2606:4700:3037::6815:1e8d, 2606:4700:3037::ac43:acfa, 172.67.172.250, ...
Connecting to www.sqlitetutorial.net (www.sqlitetutorial.net)|2606:4700:3037::6815:1e8d|:443... connected.
HTTP request sent, awaiting response... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


200 OK
Length: 305596 (298K) [application/zip]
Saving to: ‘./data/chinook.zip’

./data/chinook.zip  100%[===================>] 298.43K  --.-KB/s    in 0.07s   

2024-05-20 09:31:46 (4.30 MB/s) - ‘./data/chinook.zip’ saved [305596/305596]

Archive:  ./data/chinook.zip
replace chinook.db? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C


In [23]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine("sqlite:///chinook.db")

In [24]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine)

In [25]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    llm=llm_70b,
)

In [26]:
response = query_engine.query("What are some albums?")

print(response)

Here are some albums: For Those About To Rock We Salute You, Balls to the Wall, Restless and Wild, Let There Be Rock, Big Ones, Jagged Little Pill, Facelift, Warner 25 Anos, Plays Metallica By Four Cellos, and Audioslave.


In [27]:
response = query_engine.query("What are some artists? Limit it to 5.")

print(response)

Here are 5 artists: AC/DC, Accept, Aerosmith, Alanis Morissette, and Alice In Chains.


This last query should be a more complex join

In [28]:
response = query_engine.query(
    "What are some tracks from the artist AC/DC? Limit it to 3"
)

print(response)

Here are three tracks from the legendary Australian rock band AC/DC: "For Those About To Rock (We Salute You)", "Put The Finger On You", and "Let's Get It Up".


In [29]:
print(response.metadata["sql_query"])

SELECT tracks.Name FROM tracks INNER JOIN albums ON tracks.AlbumId = albums.AlbumId INNER JOIN artists ON albums.ArtistId = artists.ArtistId WHERE artists.Name = 'AC/DC' LIMIT 3;


## 5. Structured Data Extraction

An important use case for function calling is extracting structured objects. LlamaIndex provides an intuitive interface for this through `structured_predict` - simply define the target Pydantic class (can be nested), and given a prompt, we extract out the desired object.

**NOTE**: Since there's no native function calling support with Llama3, the structured extraction is performed by prompting the LLM + output parsing.

In [30]:
from llama_index.llms.groq import Groq
from llama_index.core.prompts import PromptTemplate
from pydantic import BaseModel


class Restaurant(BaseModel):
    """A restaurant with name, city, and cuisine."""

    name: str
    city: str
    cuisine: str


llm = Groq(model="llama3-8b-8192", pydantic_program_mode="llm")
prompt_tmpl = PromptTemplate(
    "Generate a restaurant in a given city {city_name}"
)

In [31]:
restaurant_obj = llm.structured_predict(
    Restaurant, prompt_tmpl, city_name="Miami"
)
print(restaurant_obj)

name='Café Havana' city='Miami' cuisine='Cuban'


## 6. Adding Chat History to RAG (Chat Engine)

In this section we create a stateful chatbot from a RAG pipeline, with our chat engine abstraction.

Unlike a stateless query engine, the chat engine maintains conversation history (through a memory module like buffer memory). It performs retrieval given a condensed question, and feeds the condensed question + context + chat history into the final LLM prompt.

Related resource: https://docs.llamaindex.ai/en/stable/examples/chat_engine/chat_engine_condense_plus_context/

In [32]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about the Kendrick and Drake beef."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=True,
)

In [33]:
response = chat_engine.chat(
    "Tell me about the songs Drake released in the beef."
)
print(str(response))

Condensed question: Tell me about the songs Drake released in the beef.
Context: page_label: 31
file_path: data/drake_kendrick_beef.pdf

Culture
Shaboo zey’s Cowboy Carter Features Were Only the Be ginning
By Heven Haile
Sign up for Manual, our new flagship newsletter
Useful advice on style, health, and more, four days a week.
5/10/24, 10:08 PM The Kendrick Lamar/Drake Beef, Explained | GQ
https://www.gq.com/story/the-kendrick-lamar-drake-beef-explained 31/34

page_label: 18
file_path: data/drake_kendrick_beef.pdf

Kurrco
@Kurrco·Follow
KENDRICK LAMAR
6 16 IN LA
(DRAKE DISS)
OUT NOW 
This video has been deleted.
6 08 AM · May 3, 2024
59.3K Reply Copy link
Read 1.3K replies
After all this talk about “the clock,” who among us expected Kendrick to follow up his
own titanic diss track with another missile just three days later? Friday morning he
released “6:16 in LA,” with its title of course being a nod to Drake's series of time-stamp-
Sign up for Manual, our new flagship newsletter
Usefu

In [34]:
response = chat_engine.chat("What about Kendrick?")
print(str(response))

Condensed question: What do you want to know about Kendrick Lamar's involvement in the Drake beef?
Context: page_label: 17
file_path: data/drake_kendrick_beef.pdf

Melly is, of course, the Florida rapper whose rising career came to a screeching halt
thanks to a still ongoing murder trial accusing Melly of the premeditated murders of two
YNW associates—ostensibly, two close friends. (Second best line: using Haley Joel
Osment's IMDb for a two-for-one A.I. and ghostwriters reference.)
With lines referencing Puff Daddy notoriously slapping Drake and calling out Drake's
right-hand enforcer Chubbs by name, Kendrick's threatening to “take it there,” but for
now it remains a fun war of words and one that doesn't seem likely to end anytime soon,
much less in an anticlimax like the Drake-Pusha T beef. Drake can only have been
desperate for Kendrick to respond because he has a fully loaded clip waiting to shoot,
and Kendrick for his part here, promises “headshots all year, you better walk around 

## 7. Agents

Here we build agents with Llama 3. We perform RAG over simple functions as well as the documents above.

### Agents And Tools

In [35]:
from llama_index.llms.groq import Groq

llm = Groq(model="llama3-8b-8192")
llm_70b = Groq(model="llama3-70b-8192")

In [38]:
import json
from typing import Sequence, List

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.agent.openai import OpenAIAgent

import nest_asyncio

nest_asyncio.apply()

### Define Tools

In [40]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b


def divide(a: int, b: int) -> int:
    """Divides two integers and returns the result integer"""
    return a / b


multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)
divide_tool = FunctionTool.from_defaults(fn=divide)
llm_70b.is_function_calling_model = True

### ReAct Agent

In [41]:
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool, subtract_tool, divide_tool],
    llm=llm_70b,
    verbose=True,
)

### Querying

In [42]:
response = agent.chat("What is (121 + 2) * 5?")
print(str(response))

Added user message to memory: What is (121 + 2) * 5?
=== Calling Function ===
Calling function: add with args: {"a":121,"b":2}
Got output: 123

=== Calling Function ===
Calling function: multiply with args: {"a":123,"b":5}
Got output: 615

The answer is 615.


### ReAct Agent With RAG QueryEngine Tools

In [68]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

### Create ReAct Agent using RAG QueryEngine Tools

This may take 4 minutes to run:

In [69]:
drake_index = VectorStoreIndex.from_documents(docs_drake)
drake_query_engine = drake_index.as_query_engine(similarity_top_k=3)

kendrick_index = VectorStoreIndex.from_documents(docs_kendrick)
kendrick_query_engine = kendrick_index.as_query_engine(similarity_top_k=3)

In [70]:
drake_tool = QueryEngineTool(
    drake_index.as_query_engine(),
    metadata=ToolMetadata(
        name="drake_search",
        description="Useful for searching over Drake's life.",
    ),
)

kendrick_tool = QueryEngineTool(
    kendrick_index.as_query_engine(),
    metadata=ToolMetadata(
        name="kendrick_search",
        description="Useful for searching over Kendrick's life.",
    ),
)

query_engine_tools = [drake_tool, kendrick_tool]

In [71]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm_70b,
    verbose=True,
)

### Querying

In [72]:
response = agent.chat("Tell me about how Kendrick and Drake grew up")
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: kendrick_search
Action Input: {'input': "Kendrick Lamar's childhood"}
Observation: Kendrick Lamar was born on June 17, 1987, in Compton, California. His parents, Kenneth "Kenny" Duckworth and Paul Oliver, relocated to Compton in 1984 due to his father's affiliation with the Gangster Disciples. Lamar was named after singer-songwriter Eddie Kendricks of the Temptations. He was an only child until the age of seven and was described as a loner by his mother. Eventually, his parents had his two younger brothers and younger sister, businesswoman Kayla Sawyer (née Duckworth).
Thought: I need to use a tool to help me answer the question.
Action: drake_search
Action Input: {'input': "Drake's childhood"}
Observation: Drake was raised in two neighborhoods. He lived on Weston Road in Toronto's working-class west end until grade six and attended Weston Memorial Junior Public School until grade four.
Thought: I need to use a tool 